In [3]:
# procurar e colocar créditos aqui com o link do artigo

import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Obtem os vencimentos
def vencimentos(ticker):
    url = f'https://opcoes.net.br/listaopcoes/completa?au=False&uinhc=0&idLista=ML&idAcao={ticker}&listarVencimentos=true&cotacoes=true'
    response = requests.get(url, verify=False).json()
    vctos = [[i['value'], i['text']] for i in response['data']['vencimentos']]
    return vctos

# Obtem as opções
def listar_opcoes(ticker):
    # Busca os vencimentos
    vctos = vencimentos(ticker)
    opcs=[]
    
    # Busca as opções para cada data de vencimento
    for vcto in vctos:
        url=f'https://opcoes.net.br/listaopcoes/completa?au=False&uinhc=0&idLista=ML&idAcao={ticker}&listarVencimentos=false&cotacoes=true&vencimentos={vcto[0]}'
        response = requests.get(url).json()
        
        # Busca apenas os campos necessários
        opcs += ([[ticker]+[i[0][:i[0].find('_')]] + i[2:4] + [vcto[1]] + [i[5]] + [i[8]] for i in response['data']['cotacoesOpcoes']])
    
    # Gera o dataframe
    colunas = ['ATIVO_OBJ','ATIVO', 'TIPO', 'MOD', 'DT_VCTO', 'STRIKE', 'PRECO']
    opcs = pd.DataFrame(opcs, columns=colunas)
    
    # transforma o campo vencimento e datetime
    opcs['DT_VCTO'] = pd.to_datetime(opcs['DT_VCTO'])
    
    return opcs

In [5]:
listar_opcoes('WEGE3')

,ATIVO_OBJ,ATIVO,TIPO,MOD,DT_VCTO,STRIKE,PRECO
0,WEGE3,WEGEL208,CALL,A,2022-12-16,20.83,18.00
1,WEGE3,WEGEL213,CALL,A,2022-12-16,21.33,NaN
2,WEGE3,WEGEL218,CALL,A,2022-12-16,21.83,NaN
3,WEGE3,WEGEL223,CALL,A,2022-12-16,22.33,16.10
4,WEGE3,WEGEL228,CALL,A,2022-12-16,22.83,16.75
...,...,...,...,...,...,...,...
533,WEGE3,WEGEL402,CALL,A,2023-12-15,40.29,7.00
534,WEGE3,WEGEX250,PUT,E,2023-12-15,24.79,0.30
535,WEGE3,WEGEX402,PUT,E,2023-12-15,40.29,NaN
536,WEGE3,WEGEI330,CALL,A,2024-09-20,33.00,NaN
